In [3]:
import os
import timeit
import numpy as np
from search import astar_search, manhattan_distance
from mazes import gen_sparses, get_mazes
from PacProblem import PacProblem

## Test Cases

For testing purposes, we generated 10 mazes using the [tool provided by classmate Gabriel Bomfim](https://gabomfim.github.io/pacman-mazegen/tetris/many.htm) in Google Classrom, which adapts the [maze generator](https://shaunlebron.github.io/pacman-mazegen/) linked in the project description. Each tile is represented by a char, where **|** and **-** are walls, **.** are foods and **o** are ghosts. We put the start and goal position in all mazes, respectively symbolized by **!** and **?**.

As this tool creates mazes fully filled with food, we thought that it would be good for comparision to also test sparse mazes, which we created by randomly removing dots in the dense ones. The variations, together with the originals, are shown below.

In [ ]:
denses, sparses = get_mazes()

## A* Search Solution

As an informed search algorithm, A\* takes into account information about the path cost together with an heuristic to evaluate which is the most promising path to take when it enters a state. For this evaluation, A\* chooses in state $n$ to proceed to the neighbor that gives the lowest $f(n) = g(n) + h(n)$, $g(n)$ being the exact path cost from starting state to $n$ and $h(n)$ the heuristic estimated cost from $n$ to goal state.

### Heuristic

How fast the agent reaches the goal in A\* depends highly on the heuristic implemented and how it affects the nodes expansion. In a problem like ours, where the maze configuration needs to be known in state $n$ so $f(n)$ can be calculated, it's specially important to pick a good heuristic - after all, the search space is exponentially large, as each subset of eaten food represents a different node, even with the agent in a fixed position.

A common approach to simpler pathfinding problems is to use the **Manhattan distance** as a heuristic, which is the distance between the agent and the goal positions measured along axes at right angles (i.e., $|x_1 - x_2| + |y_1 - y_2|$, given that the agent is in $(x_1, y_1)$ and the goal is to reach $(x_2, y_2)$). In our problem, however, is not realistic to estimate a good cost to goal using this distance, as the foods over the maze can make it very far from optimal.

Considering that, we implement the sum of Manhattan distances between the agent and all the foods as a heuristic, as it's highly possible that they will all be eaten in the optimal path. Notice that this sum can overestimate the optimal, because it's not always true that all the foods will be eaten in the best path. As a overestimating heuristic, it breaks admissibility - that is, A\* is not guaranteed to find the optimal path. Even so, as our problem gives a high score to Pac-Man when it eats, we chose it expecting A\* will find good, if not optimal, paths in reasonable running times. 


In [ ]:
def astar_heuristic(node):
    ''' sum of manhattan distances between Pac-Man and all foods in maze '''
    
    # Detach maze configuration and Pac-Man position
    tuple_maze, idx = node.state
    
    # Accumulate sum of manhattan distances to foods
    md_sum = 0
    for food_idx in np.argwhere(maze == '.'):
        md_sum += manhattan_distance(food_idx, idx)
            
    return md_sum

## WIP (Testing)


In [ ]:
# TODO: Script to run all tests and identify start + goal
# automatically

# Big maze 2
maze = []
for line in open('./mazes/big/2', 'r').readlines():
    # Remove trailing new lines
    row = list(line.rstrip('\n'))
    
    # Erase ! and ? chars
    row = [' ' if (c == '!' or c == '?') else c for c in row]
    
    maze.append(row)

# Maze as a tuple of tuples
maze = tuple(map(tuple, maze))
print('\n'.join(map(str, maze)))

start = (1, 1) # ! in file
goal = (1, 15) # ? in file

In [ ]:
%%time

# Problem with maze in state
problem = PacProblem((maze, start), goal)
sol = astar_search(problem, astar_heuristic)

print('Actions: ', sol.solution())
print('Score: ', -1*sol.path_cost)

In [ ]:
# Adapting heuristic for testing with no maze;
# probably there is a better way to do this
from SearchAgent import SearchAgent
from PacProblemNoMaze import PacProblem

def h(node):
    ''' sum of manhattan distances between Pac-Man and all foods in maze '''

    # Detach maze configuration and Pac-Man position
    idx = node.state
    
    # Accumulate sum of manhattan distances to foods
    md_sum = 0
    for food_idx in np.argwhere(maze == '.'):
        md_sum += manhattan_distance(food_idx, idx)
            
    return md_sum

In [ ]:
%%time

# Problem without maze in state
maze = np.array(maze).astype('bytes')
problem = PacProblem(start, goal, maze)
sol = astar_search(problem, h)

print('Actions: ', sol.solution())
print('Score: ', -1*sol.path_cost)